In [ ]:
from snowflake.snowpark import Session
connection_parameters = {
  "account": "MQJIBNP-VM41968",
  "user": "NIENKEG",
  "password": ""
}

new_session = Session.builder.configs(connection_parameters).create()

In [11]:
df = new_session.sql("use database AWS;")
df.show()

------------------------------------
|"status"                          |
------------------------------------
|Statement executed successfully.  |
------------------------------------



In [12]:
new_session.sql("""CREATE OR REPLACE STORAGE INTEGRATION my_s3_integrations
TYPE = EXTERNAL_STAGE
STORAGE_PROVIDER = S3
ENABLED = TRUE
STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::756124510499:role/connectandinspire'
STORAGE_ALLOWED_LOCATIONS = ('s3://data4enexis2usecase/cards_data.csv', 's3://data4enexis2usecase/users_data.csv',
                 's3://data4enexis2usecase/mcc_codes.json', 's3://data4enexis2usecase/train_fraud_labels.json');
""").collect()

# my_s3_integrations replaced by ENEXIS_EXTERNAL_SI

[Row(status='Integration MY_S3_INTEGRATIONS successfully created.')]

In [13]:
new_session.sql("""DESC INTEGRATION my_s3_integrations;""").collect()

[Row(property='ENABLED', property_type='Boolean', property_value='true', property_default='false'),
 Row(property='STORAGE_PROVIDER', property_type='String', property_value='S3', property_default=''),
 Row(property='STORAGE_ALLOWED_LOCATIONS', property_type='List', property_value='s3://data4enexis2usecase/cards_data.csv,s3://data4enexis2usecase/users_data.csv,s3://data4enexis2usecase/mcc_codes.json,s3://data4enexis2usecase/train_fraud_labels.json', property_default='[]'),
 Row(property='STORAGE_BLOCKED_LOCATIONS', property_type='List', property_value='', property_default='[]'),
 Row(property='STORAGE_AWS_IAM_USER_ARN', property_type='String', property_value='arn:aws:iam::296062579650:user/iovx0000-s', property_default=''),
 Row(property='STORAGE_AWS_ROLE_ARN', property_type='String', property_value='arn:aws:iam::756124510499:role/connectandinspire', property_default=''),
 Row(property='STORAGE_AWS_EXTERNAL_ID', property_type='String', property_value='VV45568_SFCRole=3_NObdHDXAspfNBNSq9

In [14]:
# cards_data.csv

new_session.sql("""
            CREATE OR REPLACE STAGE aws_s3_cards_stage
            URL = 's3://data4enexis2usecase/cards_data.csv'
            STORAGE_INTEGRATION = my_s3_integrations
            FILE_FORMAT = (TYPE = 'CSV');
            """).collect()

print("The S3 storage for the cards_data.csv is created.")

The S3 storage for the cards_data.csv is created.


In [15]:
# users_data.csv

new_session.sql("""
            CREATE OR REPLACE STAGE aws_s3_users_stage
            URL = 's3://data4enexis2usecase/users_data.csv'
            STORAGE_INTEGRATION = my_s3_integrations
            FILE_FORMAT = (TYPE = 'CSV');
            """).collect()

print("The S3 storage for the users_data.cs is created.")

The S3 storage for the users_data.cs is created.


In [16]:
# mcc_codes.json

new_session.sql("""
            CREATE OR REPLACE STAGE aws_s3_mcc_stage
            URL = 's3://data4enexis2usecase/mcc_codes.json'
            STORAGE_INTEGRATION = my_s3_integrations
            FILE_FORMAT = (TYPE = 'JSON');
            """).collect()

print("The S3 storage for the mcc_codes.json is created.")

The S3 storage for the mcc_codes.json is created.


In [17]:
# train_fraud_labels.json

new_session.sql("""
            CREATE OR REPLACE STAGE aws_s3_fraud_stage
            URL = 's3://data4enexis2usecase/train_fraud_labels.json'
            STORAGE_INTEGRATION = my_s3_integrations
            FILE_FORMAT = (TYPE = 'JSON');
            """).collect()

print("The S3 storage for the train_fraud_labels.json is created.")

The S3 storage for the train_fraud_labels.json is created.


In [19]:
new_session.sql("""LIST @my_s3_integrations;""")

In [ ]:
# Create csv format

new_session.sql("""
        CREATE OR REPLACE FILE FORMAT mijn_csv_format
        TYPE = 'CSV'
        FIELD_OPTIONALLY_ENCLOSED_BY = '"'
        SKIP_HEADER = 1
        FIELD_DELIMITER = ',';
            """).collect()

print("CSV format aangemaakt")

CSV format aangemaakt


In [25]:
# Create table 

# has_chip and card_on_dark_web should be boolean
new_session.sql("""
                CREATE OR REPLACE TABLE cards_data (
                id INT,
                client_id INT,
                card_brand STRING,
                card_type Varchar(20),
                card_number INT,
                expires DATE,
                cvv INT,
                has_chip varchar(10),
                num_cards_issued INT,
                credit_limit varchar(30),
                acct_open_date DATE,
                year_pin_last_changed DATE,
                card_on_dark_web varchar(10));
            """).collect()


print("Tabel cards_data aangemaakt")


Tabel cards_data aangemaakt


In [27]:
# Vul tabel cards_data

new_session.sql("""
                COPY INTO cards_data
                FROM @aws_s3_cards_stage
                FILE_FORMAT = (FORMAT_NAME = 'mijn_csv_format');
                """).collect()

print("Tabel cards_data gevuld")



SnowparkSQLException: (1304): 01bc8949-0106-9877-0002-b46a0009508a: 003167 (42601): Error assuming AWS_ROLE:
User: arn:aws:iam::296062579650:user/iovx0000-s is not authorized to perform: sts:AssumeRole on resource: arn:aws:iam::756124510499:role/connectandinspire